In [6]:
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset  # 数据集对象
from torch.utils.data import DataLoader # 数据加载器
import torch.nn as nn   # 神经网络模块，实现各种网络层
import torch.optim as optim # 优化器模块，实现各种优化器，比如SGD,Adam/AdamW
from sklearn.model_selection import train_test_split    # 划分数据集
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from torchsummary import summary

训练数据集,测试数据集准备

In [21]:
def dataset():
    # 读取csv
    df = pd.read_csv('./data/手机价格预测.csv')
    # 切分特征标签
    x = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    # 划分训练集测试集
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=28,stratify=y)
    # 数据标准化
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    # 封装数据集
    train_dataset = TensorDataset(torch.tensor(x_train).to(dtype=torch.float32),torch.tensor(y_train.values).to(dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(x_test).to(dtype=torch.float32),torch.tensor(y_test.values).to(dtype=torch.long))
    return train_dataset,test_dataset

模型搭建

In [26]:
class Model(nn.Module):
    def __init__(self):
        # 构建神经元
        super().__init__()
        # 20个特征输入,256个输出
        self.linear1 = nn.Linear(20,256)
        self.linear2 = nn.Linear(256,256)
        # 256个输入,4个标签输出
        self.linear3 = nn.Linear(256,4)
        # dropout层,死亡概率0.1
        self.dropout = nn.Dropout(0.1)
    # 前向传播
    def forward(self,x):
        # 第一层神经元激活函数ReLU
        x = torch.relu(self.linear1(x))
        # dropout层随机失活
        x = self.dropout(x)
        x = torch.relu(self.linear2(x))
        x = self.dropout(x)
        # 最后哟哦那个多分类交叉熵不需要softmax层
        x = self.linear3(x)
        return x

模型训练

In [42]:
def train(train_dataset,epochs):
    # 准备模型
    model = Model()
    # 用gpu训练
    model = model.to(device='cuda')
    # 设置数据加载器(训练数据集,训练尺寸,是否打乱数据,是否删除最后一批训练批次)
    data_loder = DataLoader(train_dataset,batch_size=32,shuffle=True,drop_last=False)
    # 准备损失函数
    loss_fn = nn.CrossEntropyLoss()
    # 优化策略,这里使用adamw
    optimizer = optim.AdamW(model.parameters(),lr=1e-3)
    # 学习率调度:这里使用周期重启的余弦退火策略
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        # 传入优化器
        optimizer,
        # 训练多少轮次进行一次重启
        T_0=50,
        # 衰减最小学习率
        eta_min=0
    )
    # 寻连epochs轮
    for i in range(epochs):
        # 开启训练模式
        model.train()
        # 初始化本轮总损失,本轮正确率,本轮时间,总样本数
        train_loss = 0.0
        train_acc = 0.0
        total_samples = 0
        # 记录开始时间
        start = time.time()
        for x,y in data_loder:
            x = x.to(device='cuda')
            y = y.to(device='cuda')
            # 前向传播
            y_pred = model(x)
            # 计算损失
            loss = loss_fn(y_pred,y)
            # 梯度清零
            optimizer.zero_grad()
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            # 计算本批次总损失以供计算本轮平均损失
            # 先获取批次样本数
            samples = x.shape[0]
            # 每个批次损失累加获得总损失
            train_loss += loss*samples
            # 每个批次预测正确的样本个数累加获得本轮训练预测正确的样本个数
            train_acc += (y_pred.argmax(dim=-1)==y).sum().item()
            # 获取总样本数
            total_samples += samples
        # 学习率调度
        scheduler.step()
        # 计算本轮平均损失
        loss_mean = train_loss/total_samples
        # 计算本轮正确率
        acc = train_acc/total_samples
        # 本轮训练时间
        end = time.time()
        print(f'{i+1}/{epochs}|训练损失{loss_mean}|正确率|{acc}|本轮训练时间{start-end}s')
        # 创建保存模型的文件夹,如果已存在则不爆错
    os.makedirs("model", exist_ok=True)
    # 保存训练好的模型参数
    torch.save(model.state_dict(), "model/phone_price_classifier.pth")

模型测试

In [39]:
def test(test_dataset):
    # 准备模型
    model = Model()
    # 用gpu训练
    model = model.to(device='cuda')
    # 加载模型参数
    model.load_state_dict(torch.load("model/phone_price_classifier.pth"))
    # 设置数据加载器(训练数据集,训练尺寸,是否打乱数据(测试不打乱数据),是否删除最后一批训练批次)
    data_loder = DataLoader(test_dataset,batch_size=32,shuffle=False,drop_last=False)
    # 准备损失函数
    loss_fn = nn.CrossEntropyLoss()
    # 开启测试模式(关闭dropout)
    model.eval()
    # 初始化本轮总损失,本轮正确率,本轮时间,总样本数
    train_loss = 0.0
    train_acc = 0.0
    total_samples = 0
    # 记录开始时间
    start = time.time()
    for x,y in data_loder:
        x = x.to(device='cuda')
        y = y.to(device='cuda')
        # 前向传播
        y_pred = model(x)
        # 计算损失
        loss = loss_fn(y_pred,y)
        samples = x.shape[0]
        # 每个批次损失累加获得总损失
        train_loss += loss*samples
        # 每个批次预测正确的样本个数累加获得本轮训练预测正确的样本个数
        train_acc += (y_pred.argmax(dim=-1)==y).sum().item()
        # 获取总样本数
        total_samples += samples
    # 计算本轮平均损失
    loss_mean = train_loss/total_samples
    # 计算本轮正确率
    acc = train_acc/total_samples
    # 本轮训练时间
    end = time.time()
    print(f'|损失{loss_mean}|正确率|{acc}|预测时间{start-end}s')

测试

In [43]:
# 测试
if __name__ == "__main__":
    train_dataset, test_dataset=dataset()
    print(f"训练集数量：{len(train_dataset)}")
    print(f"测试集数量：{len(test_dataset)}")
    print(f"train_dataset:{train_dataset}, test_dataset:{test_dataset}")
    train(train_dataset,epochs=100)
    test(test_dataset)

训练集数量：1600
测试集数量：400
train_dataset:<torch.utils.data.dataset.TensorDataset object at 0x00000248EB846DA0>, test_dataset:<torch.utils.data.dataset.TensorDataset object at 0x00000248EB846A40>
1/100|训练损失1.0118144750595093|正确率|0.600625|本轮训练时间-0.17152690887451172s
2/100|训练损失0.4467993974685669|正确率|0.8475|本轮训练时间-0.14772486686706543s
3/100|训练损失0.2720784842967987|正确率|0.909375|本轮训练时间-0.14296340942382812s
4/100|训练损失0.20435401797294617|正确率|0.934375|本轮训练时间-0.14627671241760254s
5/100|训练损失0.1507672220468521|正确率|0.951875|本轮训练时间-0.14522075653076172s
6/100|训练损失0.13293933868408203|正确率|0.95|本轮训练时间-0.1467878818511963s
7/100|训练损失0.11209991574287415|正确率|0.955|本轮训练时间-0.15517354011535645s
8/100|训练损失0.0956183448433876|正确率|0.96875|本轮训练时间-0.14912033081054688s
9/100|训练损失0.09263903647661209|正确率|0.963125|本轮训练时间-0.1477959156036377s
10/100|训练损失0.06227251887321472|正确率|0.984375|本轮训练时间-0.14394354820251465s
11/100|训练损失0.0620914064347744|正确率|0.978125|本轮训练时间-0.14806151390075684s
12/100|训练损失0.06388366222381592|正确率|0.98125|本轮训